In [1]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': '192.168.178.43','port': 9200}])
es

<Elasticsearch([{'host': '192.168.178.43', 'port': 9200}])>

In [2]:
import requests
res = requests.get('http://nuc:9200')
print(res.content)

b'{\n  "name" : "nuc",\n  "cluster_name" : "elasticsearch",\n  "cluster_uuid" : "XaVn2UyNR0OD5ymUFyxvbw",\n  "version" : {\n    "number" : "7.1.1",\n    "build_flavor" : "default",\n    "build_type" : "tar",\n    "build_hash" : "7a013de",\n    "build_date" : "2019-05-23T14:04:00.380842Z",\n    "build_snapshot" : false,\n    "lucene_version" : "8.0.0",\n    "minimum_wire_compatibility_version" : "6.8.0",\n    "minimum_index_compatibility_version" : "6.0.0-beta1"\n  },\n  "tagline" : "You Know, for Search"\n}\n'


In [ ]:
import json
r = requests.get('http://nuc:9200') 
i = 1
while r.status_code == 200:
    r = requests.get('http://swapi.co/api/people/'+ str(i))
    es.index(index='sw', doc_type='people', id=i, body=json.loads(r.content))
    i=i+1
 
print(i)

In [ ]:
e1={
    "file_name":"n_x_y_z",
    "project":"CMIP6",
    "size": 27098,
    "pid": "a:alsjfalsjflsajf",
    "collection": ['n_x_y'],
}

In [ ]:
res = es.index(index='cmip6',doc_type='file',id=1,body=e1)

In [ ]:
es.get(index='sw', doc_type='people', id=5)

In [5]:
#%%time
import os
import itertools
import multiprocessing
import xarray as xr
from netCDF4 import Dataset

ATTR = {}
DIR_PATTERN = {}
FILE_PATTERN = {}
DIR_PATTERN['CMIP5']=['activity','product','institute','model','experiment','frequency','realm','variable','ensemble','lfile']
FILE_PATTERN['CMIP5'] = ['variable','table','model','experiment','ensemble','time']
DIR_PATTERN['CMIP6'] = ['mip_era','activity','institution','model','experiment','member','table','variable','grid','version','lfile']
FILE_PATTERN['CMIP6'] = ['variable','table','model','experiment','member','grid','time']

ATTR['CMIP6'] = ['tracking_id','contact']
def match(proj,path,file):
    p_dict = {}
    f_dict = {}  
    file_name = file.split('.')[0]
    f_parts = file_name.split('_')
    f_dict = dict(zip(FILE_PATTERN[proj],f_parts))
    return f_dict
  
def get_md(type,proj,root,file):
    ratts={}
    path = os.path.join(root,file)
    if type =="xarray":      
        data = xr.open_dataset(path, decode_times=False)
        attrs = data.attrs
        for key in ATTR[proj]:
            ratts[key]=attrs[key]
        data.close()    
    if type =="netcdf4":
        data = Dataset(path,mode='r')
        for key in ATTR[proj]:
            ratts[key]=getattr(data,key)
        data.close()    
                
    return(ratts)

def worker(proj,root,dirs,files):
    
    tdict={}
    for mfile in files: 
        tdict[mfile]={}
        tdict[mfile] = match(proj,root,mfile)
        tdict[mfile]['file_name']=mfile
        tdict[mfile]['project']=proj
        #ratts = get_md("netcdf4",proj,root,mfile)
        #for key,val in ratts.items():
            #tdict[mfile][key]=val
    if len(files)>0:
        tdict[mfile]['dataset_id']=root 
        tt = tdict[mfile]['time'].split("-")
        tdict[mfile]['stime']=tt[0]
        tdict[mfile]['etime']=tt[1]
    return tdict
        
walk = os.walk("/run/media/stephan/Volume/data")   

for root,dirs,files in walk:
    print("=============================")
    res = worker('CMIP6',root,dirs,files)   
    # update elastic search
    i = 0
    for item,i_dict in res.items():
        i+=1
        res = es.index(index='cmip6',id=i,body=i_dict)
    print("============================")

In [ ]:
res = es.get(index='cmip6',id=1)
print(res)

In [43]:
q_string = {"query": {
               "bool": {
                   "must": [
                       {"match": {"variable": "tas"}},
                       {"match": {"model":"GFDL-CM4"}},
                       {"match": {"time":'015101-025012'}}
                                  ]             
                      }
               }
           }
r_string = {"query": {
               "bool": {
                   "must": [
                       {"match": {"stime": "055101"}},
                       {"match": {"model":"GFDL-CM4"}}                   
                                  ]             
                      }
               }
           }

r_string = {"query": {
               "bool": {
                   "filter": [
                       {"term": {"stime":"055101"}}
                   ]
                      }
               }
           }

t_string = {"query": {
              "filter" : {
                 "term" : { "stime": "015101"}
                }
            }}

In [44]:
import pprint
pp=pprint.PrettyPrinter(indent=4)
res = es.search(index="cmip6", body=r_string)
pp.pprint(res['hits']['hits'])


[   {   '_id': '17',
        '_index': 'cmip6',
        '_score': 0.0,
        '_source': {   'contact': 'gfdl.climate.model.info@noaa.gov',
                       'dataset_id': '/run/media/stephan/Volume/data/tas',
                       'etime': '065012',
                       'experiment': 'piControl',
                       'file_name': 'tas_Amon_GFDL-CM4_piControl_r1i1p1f1_gr1_055101-065012.nc',
                       'grid': 'gr1',
                       'member': 'r1i1p1f1',
                       'model': 'GFDL-CM4',
                       'project': 'CMIP6',
                       'stime': '055101',
                       'table': 'Amon',
                       'time': '055101-065012',
                       'tracking_id': 'hdl:21.14100/c2669cfe-c044-4a86-a1e3-7526466bf145',
                       'variable': 'tas'},
        '_type': 'file'},
    {   '_id': '5',
        '_index': 'cmip6',
        '_score': 0.0,
        '_source': {   'contact': 'gfdl.climate.model.info@noaa.g

In [ ]:
t_d = {}
t_d1 = {'a':'b'}
print(len(t_d))
print(len(t_d1))

In [ ]:
import itertools
import multiprocessing

def worker(filename,dset):
    print(filename,dset)
      # do something here!

def main():
    with multiprocessing.Pool(2) as Pool: # pool of 48 processes

        walk = os.walk("/run/media/stephan/Volume/data")
        fn_gen = itertools.chain.from_iterable((os.path.join(root, file)
                                                for file in files)
                                                for root, dirs, files in walk)

        results_of_work = pool.map(worker, fn_gen) # this does the parallel processin

In [ ]:
!pip install elasticsearch-dsl


In [ ]:
from elasticsearch_dsl import Document, Date, Text, Keyword, Boolean, InnerDoc, \
Nested, Object, Join, Long, connections

In [12]:
from elasticsearch_dsl import Search
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': '192.168.178.43','port': 9200}])

s = Search(using=es,index="cmip6").filter("term",stime="015101")

In [13]:
response = s.execute()

In [15]:
for hit in response:
    print(hit.file_name)

In [30]:
import xarray as xr
from netCDF4 import Dataset

file = "/run/media/stephan/Volume/data/hus/hus_day_IPSL-CM6A-LR_abrupt-4xCO2_r10i1p1f1_gr_18501001-18541231.nc"
tst = Dataset(file,format="NETCDF4")

In [31]:
tst.data_model

'NETCDF4'

In [32]:
tst.ncattrs()

['name',
 'Conventions',
 'creation_date',
 'tracking_id',
 'description',
 'title',
 'activity_id',
 'contact',
 'data_specs_version',
 'dr2xml_version',
 'experiment_id',
 'experiment',
 'external_variables',
 'forcing_index',
 'frequency',
 'further_info_url',
 'grid',
 'grid_label',
 'nominal_resolution',
 'initialization_index',
 'institution_id',
 'institution',
 'license',
 'mip_era',
 'parent_experiment_id',
 'parent_mip_era',
 'parent_activity_id',
 'parent_source_id',
 'parent_time_units',
 'branch_method',
 'branch_time_in_parent',
 'branch_time_in_child',
 'physics_index',
 'product',
 'realization_index',
 'realm',
 'source',
 'source_id',
 'source_type',
 'sub_experiment_id',
 'sub_experiment',
 'table_id',
 'variable_id',
 'variant_label',
 'EXPID',
 'CMIP6_CV_version',
 'dr2xml_md5sum',
 'model_version',
 'history',
 'parent_variant_label',
 'NCO',
 'nco_openmp_thread_number']

In [35]:
getattr(tst,'contact')

'ipsl-cmip6@listes.ipsl.fr'

In [36]:
tst.contact

'ipsl-cmip6@listes.ipsl.fr'